In [38]:
import pytesseract as ocr
import glob  
import os
from PIL import Image
from tqdm import tqdm 
from datetime import date
from pdf2image import convert_from_path 
from pdf2image.exceptions import (PDFInfoNotInstalledError,PDFPageCountError,PDFSyntaxError)

In [41]:
#Função para OCR de arquivo pdf

def ocr_pdf(arquivo_pdf):

    '''Função que recebe um arquivo PDF e devolve uma string com o OCR do conteúdo'''

    nome_do_arquivo = os.path.split(arquivo_pdf)[1]

    cabecalho = 50*'*' + f' Conteúdo {nome_do_arquivo} ' + 50*'*' + '\n'

    #print(f'Inicio OCR do arquivo {nome_do_arquivo}')

    try:
        imagem_paginas = convert_from_path(arquivo_pdf) 
    except PDFInfoNotInstalledError:
        erro = cabecalho + 'Arquivo com problema. Erro: PDFInfoNotInstalledError.' + '\n'
        return erro
    except PDFPageCountError:
        erro = cabecalho + 'Arquivo com problema. Erro: PDFPageCountError.' + '\n'
        return erro
    except PDFSyntaxError:
        erro = cabecalho + 'Arquivo com problema. Erro: PDFSyntaxError.' + '\n'
        return erro

    #Convertendo pdf para imagens
    for i,pagina in enumerate(imagem_paginas):
        pagina.save(f'pagina_' + str(i) + '.jpg', 'JPEG')       

    #OCR das imagens
    files = glob.glob('*.jpg')
    paginas = []
    for filename in files:
        t = ocr.image_to_string(Image.open(filename), lang='por')
        #tratando o texto
        t = t.replace('\\n \\n','').replace('\\n\\n','').replace('\\n',' ').replace('   ',' ').replace('    ','') + '\n'
        t = t.split() 
        t = ' '.join(t)
        paginas.append(t)

    #Apagando arquivos de imagens
    for py_file in files:
        try:
            os.remove(py_file)
        except OSError as e:
            print(f"Error:{e.strerror}")
    
    #print('Fim')   
        
    texto = cabecalho + str(paginas).replace(']','').replace('[','') + '\n'

    return texto

In [42]:
#pasta com arquivos pdf
arquivos_pdf = glob.glob('arquivos/*.pdf')

data_atual = date.today().strftime('%d-%m-%Y')

for pdf in tqdm(arquivos_pdf):

    #Chama função que faz o OCR
    texto_pdfs = ocr_pdf(pdf)

    #Gravando resultado em um arquivo de texto 
    with open(f'OCR-{data_atual}.txt', 'a') as temp_file:
            temp_file.write(texto_pdfs)
            temp_file.write('\n')
    
    texto_pdfs = ''

print('Fim dos arquivos')

100%|██████████| 2/2 [00:44<00:00, 22.44s/it]

Fim dos arquivos
